2. Indicar si existe algún tipo de Homofilia y qué tipo de homofilia es. Si no hay homofilia por ningún criterio, explicar. Justificar detalladamente.
    
    **[3 puntos]**

In [46]:
import networkx as nx
import pandas as pd
import sys
sys.path.append('../')
from social_networks_utils.homofilia import proporcion_cruzan_campo

In [47]:
net = pd.read_csv('../data/World.csv')
net.head()

,Origen,Destino
0,Papua New Guinea,Australia
1,Papua New Guinea,Philippines
2,Papua New Guinea,Indonesia
3,Papua New Guinea,Solomon Islands
4,Papua New Guinea,Hong Kong


In [48]:
G = nx.from_pandas_edgelist(net,'Origen','Destino')

Saque la info de paises y continentes de https://gist.github.com/canfixit/1662664

In [49]:
import data.countries_data

In [50]:
country_to_continent_d = {}
for country in countries_data.COUNTRIES:
    country_to_continent_d[country['name']] = country['continent']

In [51]:
no_continent_countries = set()
for country in list(G.nodes):
    country_to_continent_d[country] = country_to_continent_d.get(country,'None')
    if country_to_continent_d[country] == 'None':
        no_continent_countries.add(country)

In [52]:
no_continent_countries

set()

Después de haber completado COUNTRIES, tenemos todas las locaciones asignadas a su respectivo continente.

In [53]:
prob = {
   'Oceania': 0,
    'Asia': 0,
    'North America': 0,
    'Europe': 0,
    'South America': 0,
    'Africa': 0
}

for country in list(G.nodes):
    if country_to_continent_d[country] == 'Oceania':
        prob['Oceania'] += 1
    elif country_to_continent_d[country] == 'Asia':
        prob['Asia'] += 1
    elif country_to_continent_d[country] == 'North America':
        prob['North America'] += 1
    elif country_to_continent_d[country] == 'Europe':
        prob['Europe'] += 1
    elif country_to_continent_d[country] == 'South America':
        prob['South America'] += 1
    elif country_to_continent_d[country] == 'Africa':
        prob['Africa'] += 1

prob.update((x,y/len(G)) for x,y in prob.items())

Me aseguro que la suma de todas las probabilidades sea igual a 1

In [54]:
sum(prob.values())

1.0

In [55]:
prob_totales = {
    'Oceania': 
    {
        'Interbloque': prob['Oceania']**2,
        'Cruzada': prob['Oceania']*(1-prob['Oceania'])
    },
    'Asia': 
    {
        'Interbloque': prob['Asia']**2,
        'Cruzada': prob['Asia']*(1-prob['Asia'])
    },
    'North America': 
    {
        'Interbloque': prob['North America']**2,
        'Cruzada': prob['North America']*(1-prob['North America'])
    },
    'Europe': 
    {
        'Interbloque': prob['Europe']**2,
        'Cruzada': prob['Europe']*(1-prob['Europe'])
    },
    'South America': 
    {
        'Interbloque': prob['South America']**2,
        'Cruzada': prob['South America']*(1-prob['South America'])
    },
    'Africa': 
    {
        'Interbloque': prob['Africa']**2,
        'Cruzada': prob['Africa']*(1-prob['Africa'])
    },
}

In [56]:
def mapper_country_to_continent(country_name):
    return country_to_continent_d[country_name]

In [57]:
prop_teo_cruzar_campo = 0
for i in prob_totales:
    prop_teo_cruzar_campo += prob_totales[i]['Cruzada'] 

In [58]:
prop_real_cruzar_campo = proporcion_cruzan_campo(G,mapper_country_to_continent)

In [59]:
porcentaje_sin_homofilia = (prop_real_cruzar_campo/prop_teo_cruzar_campo)*100

In [60]:
print ('Tenemos un',porcentaje_sin_homofilia,end='% ')
print ('sin homofilia')

Tenemos un 54.98773660685799% sin homofilia


Podemos concluir que hay homofilia por continente!

![Homofilia](../media/homofilia.png)

Corriendo modularity en gephi podemos separar los paises por comunidades, y claramente se nota que cada color corresponde a un continente, salvo algunas excepciones interesantes:
* España (único país europeo hipanohablante) esta dentro de la comunidad de países de América (Azul)
* Francia (gran cantidad de afroeuropeos) esta dentro de la comunidad de países Africanos (Verde)